In [89]:
import torch
import torch.nn as nn
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as f
import torch.optim as optim

In [90]:
class LNET(nn.Module):
    def __init__(self):
        super(LNET,self).__init__()
        self.relu = nn.Tanh()
        self.pool = nn.AvgPool2d(kernel_size=(2,2) , stride=(2,2))
        self.conv1 = nn.Conv2d(in_channels=1 , out_channels=6 , kernel_size=(5,5) ,stride=(1,1) , padding=(0,0))
        self.conv2 = nn.Conv2d(in_channels=6 , out_channels=16 , kernel_size=(5,5) , stride=(1,1) ,  padding=(0,0))
        self.conv3 = nn.Conv2d(in_channels=16 ,out_channels=120 , kernel_size=(5,5) , stride=(1,1) ,padding=(0,0) )
        self.linear1=nn.Linear(120, 84)
        self.linear2 = nn.Linear(84,10)
    def forward(self,x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = x.reshape(x.size(0) , -1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [91]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [92]:
batch_size = 64
learning_rate = 0.001
num_eochs = 3

In [93]:
model = LNET().to(device=device)

In [94]:
test_data = MNIST(root='/dataset' , train=False , transform = transforms.Compose([
    transforms.Pad(2),       # 28x28 → 32x32
    transforms.ToTensor()
]),download=True)
train_data =  MNIST(root='/dataset' , train=True , transform = transforms.Compose([
    transforms.Pad(2),       # 28x28 → 32x32
    transforms.ToTensor()
])
 ,download=True)
test_loader = DataLoader(test_data , shuffle=True , batch_size=batch_size)
train_loader = DataLoader(train_data , shuffle=True , batch_size=batch_size)

In [95]:
critision = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr=learning_rate)


In [96]:
for epochs in range(num_eochs):
    for batch_idx,(data , targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        scores = model(data)
        loss = critision(scores,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [98]:
def check_accuracy(loader,model):
    if loader.dataset.train:
        print("accuracy for train dataset")
    else:
        print("accuracy for test datsets")
    num_correct = 0
    num_samples=0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device = device)
           # x =x.reshape(x.shape[0] ,-1)
            scores = model(x)
            _,predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
        print(f'got {num_correct}/{num_samples} with accuraccy  {float(num_correct)/float(num_samples)*100:.2f}'
        )
    model.train()


In [100]:
check_accuracy(train_loader,model)

accuracy for train dataset
got 59085/60000 with accuraccy  98.47


In [101]:
check_accuracy(test_loader,model)

accuracy for test datsets
got 9815/10000 with accuraccy  98.15
